In [1]:
import pandas as pd
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from tqdm.notebook import tqdm

In [ ]:
movies = pd.read_csv('D:\\Загрузки\\ml-latest\\movies.csv')
ratings = pd.read_csv('D:\\Загрузки\\ml-latest\\ratings.csv')

In [ ]:
ratings

In [ ]:
ratings = ratings.drop(['timestamp'], axis=1)
ratings

In [2]:
data = pd.read_csv('D:\\Загрузки\\ml-latest\\movies_ratings.csv')

In [3]:
data = data.drop(['Unnamed: 0'], axis=1)
data

,uId,iId,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0
...,...,...,...
1570727,15476.0,Someone (2016),2.0
1570728,11655.0,The Fencing Master (1992),3.0
1570729,4871.0,Empowered (2018),2.0
1570730,4226.0,Raazi (2018),4.0


In [26]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(data, reader)

In [12]:
trainset, testset = train_test_split(data, test_size=0.3)

In [14]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Обучение заняло 1,5 часа =_=

In [15]:
test_pred = algo.test(testset)

In [16]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8697


0.8696896874298305

Точность получилась меньше 0,87, но при такой длительности обучения невозможно проводить экспреименты по поиску  
оптимального алгоритма

Возьмем 1/5 часть исходного датасета для экспериментов

In [7]:
df = data.sample(frac=1).reset_index(drop=True)
df = df.drop(['Unnamed: 0'], axis=1)

In [8]:
df1 = df.loc[:int(len(df)/5), :]

In [9]:
df1

,uId,iId,rating
0,12671.0,Fahrenheit 451 (1966),3.5
1,6461.0,Muriel's Wedding (1994),1.0
2,9540.0,Mean Girls (2004),4.0
3,3759.0,Avatar (2009),4.5
4,6944.0,"First Wives Club, The (1996)",2.0
...,...,...,...
314142,4705.0,"Blind Side, The (2009)",4.5
314143,13135.0,Fargo (1996),3.0
314144,185.0,Philadelphia (1993),3.0
314145,9724.0,Grumpier Old Men (1995),4.0


In [10]:
reader = Reader(rating_scale=(0.5, 5))
df_sp = Dataset.load_from_df(df1, reader)
trainset, testset = train_test_split(df_sp, test_size=0.3)

In [11]:
algo1 = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo2= KNNWithMeans(k=50, sim_options={'name': 'cosine', 'user_based': True})
algo3 = KNNWithMeans(k=50, sim_options={'name': 'msd', 'user_based': True})

In [12]:
from surprise.model_selection import KFold

kfold = KFold(n_splits=5)

for trainset, testset in tqdm(kfold.split(df_sp)):
    algo1.fit(trainset)
    pred = algo1.test(testset)
    
    accuracy.rmse(pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9820
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9772
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9750
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9864
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9785



In [14]:


for trainset, testset in tqdm(kfold.split(df_sp)):
    algo2.fit(trainset)
    pred = algo2.test(testset)
    
    accuracy.rmse(pred, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9568
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9519
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9613
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9523
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9581



In [15]:
for trainset, testset in kfold.split(df_sp):
    algo3.fit(trainset)
    pred = algo3.test(testset)
    
    accuracy.rmse(pred, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9516
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9551
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9555
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9530
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9555


In [30]:
algo2.predict(uid = 185, iid='Philadelphia (1993)')
# Ошибка на 0,82
algo2.predict(uid = 4705, iid='Blind Side, The (2009)')
# Ошибка на 1

Prediction(uid=4705, iid='Blind Side, The (2009)', r_ui=None, est=3.5181940808059906, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

Ка мы видим на величину ошибки влияет только количество данных обучения, алгоритмы в данной библиотеке явл-ся  
производными алгоритмами к-ближайших соседей, и нет принципиальной разницы в выборе между ними